# Feature Based Image Alignment using OpenCV

https://learnopencv.com/image-alignment-feature-based-using-opencv-c-python/

In [1]:
from __future__ import print_function
import cv2
import numpy as np

Features are detected using **ORB (Oriented FAST and Rotated BRIEF)** features in the two images. Although we need only 4 features to compute the homography, typically hundreds of features are detected in the two images. We control the number of features using the parameter MAX_MATCHES.

ORB is a fusion of FAST keypoint detector and BRIEF descriptor with some added features to improve the performance. **FAST is Features from Accelerated Segment Test** used to detect features from the provided image. It also uses a pyramid to produce multiscale-features. Now it doesn’t compute the orientation and descriptors for the features, so this is where BRIEF comes in the role.

ORB uses BRIEF descriptors but as the BRIEF performs poorly with rotation. So what ORB does is to rotate the BRIEF according to the orientation of keypoints. Using the orientation of the patch, its rotation matrix is found and rotates the BRIEF to get the rotated version. ORB is an efficient alternative to SIFT or SURF algorithms used for feature extraction, in computation cost, matching performance, and mainly the patents. SIFT and SURF are patented and you are supposed to pay them for its use. But ORB is not patented. 

In [12]:
MAX_MATCHES = 500
GOOD_MATCH_PERCENT = 0.15

We find the matching features in the two images, sort them by goodness of match and keep only a small percentage (GOOD_MATCH_PERCENT) of original matches.

In [13]:
def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_MATCHES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
    
  matches = list(matches)  # KJK added this, as matches is a tuple, which does not have a 'sort' attribute

  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)

  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
  
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
  
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
  
  return im1Reg, h

In [14]:
if __name__ == '__main__':
  
  # Read reference image
  refFilename = "form.jpg"
  refFilename = "card.jpg"
  refFilename = "D:\imageData\LaserClean\Fiducial-pre.tif"
  print("Reading reference image : ", refFilename)
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)  # cv2.IMREAD_GRAYSCALE   cv2.IMREAD_COLOR

  small_to_large_image_size_ratio = 0.2
  imReference = cv2.resize( imReference, # original image
                       (0,0), # set fx and fy, not the final size
                       fx=small_to_large_image_size_ratio, 
                       fy=small_to_large_image_size_ratio,
                       interpolation=cv2.INTER_NEAREST)

  # Read image to be aligned
  imFilename = "scanned-form.jpg"
  imFilename = "card-scanned.jpg"
  imFilename = "D:\imageData\LaserClean\Fiducial-post.tif"
  print("Reading image to align : ", imFilename);  
  im = cv2.imread(imFilename, cv2.IMREAD_COLOR)   # cv2.IMREAD_GRAYSCALE   cv2.IMREAD_COLOR

  small_to_large_image_size_ratio = 0.2
  im = cv2.resize( im, # original image
                       (0,0), # set fx and fy, not the final size
                       fx=small_to_large_image_size_ratio, 
                       fy=small_to_large_image_size_ratio,
                       interpolation=cv2.INTER_NEAREST)
  
  print("Aligning images ...")
  # Registered image will be resotred in imReg. 
  # The estimated homography will be stored in h. 
  imReg, h = alignImages(im, imReference)
  
  # Write aligned image to disk. 
  outFilename = "card-aligned.jpg"
  print("Saving aligned image : ", outFilename); 
  cv2.imwrite(outFilename, imReg)

  # Print estimated homography
  print("Estimated homography : \n",  h)

Reading reference image :  D:\imageData\LaserClean\Fiducial-pre.tif
Reading image to align :  D:\imageData\LaserClean\Fiducial-post.tif
Aligning images ...
Saving aligned image :  card-aligned.jpg
Estimated homography : 
 [[-6.39425518e-01 -5.79866662e-02  3.67339587e+03]
 [-1.05321932e+00  5.08254370e-02  5.68297200e+03]
 [-1.85308866e-04  1.08719500e-05  1.00000000e+00]]
